# NPZD slab model

For the first model example I will need:
- Forcing input
- Forcing_fluxes:
    - mixing
    - sinking
- 4 components (single) N, P, Z, D
- Fluxes:
TODO: GO OVER THE equations! Base model structure in math!
    - Grazing - linked to all NPZD
    - PhytoplanktonGrowth
        - Light - Dependence
        - Nutrients - Dependence
        - Temp - Dependence
    - mortality
    - remineralisation
    - higher order pred, etc

In [ ]:
import numpy as np
import xsimlab as xs

In [ ]:
import os,sys,inspect
sys.path.insert(1, os.path.join(sys.path[0], '..'))

import phydra


from phydra.utility.xsimlabwrappers import phydra_setup

In [ ]:
from phydra.processes.main import Grid0D, Boundary0D
from phydra.processes.components import Component, Time, make_Component
from phydra.processes.environments import BaseEnvironment, Slab
from phydra.processes.fluxes import Flux, LimitedGrowth
from phydra.processes.forcingfluxes import Mixing, Sinking, Upwelling, make_FX_flux
from phydra.processes.forcing import SinusoidalMLD, ConstantN0
from phydra.processes.gekkocontext import GekkoContext, GekkoSolve, InheritGekkoContext

In [4]:
from phydra.processes.forcing import NutrientForcing, MLDForcing

In [5]:
print(dir())

['BaseEnvironment', 'Boundary0D', 'Component', 'ConstantN0', 'Flux', 'GekkoContext', 'GekkoSolve', 'Grid0D', 'In', 'InheritGekkoContext', 'LimitedGrowth', 'MLDForcing', 'Mixing', 'NutrientForcing', 'Out', 'Sinking', 'SinusoidalMLD', 'Slab', 'Time', 'Upwelling', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_i3', '_i4', '_i5', '_ih', '_ii', '_iii', '_oh', 'exit', 'get_ipython', 'inspect', 'make_Component', 'make_FX_flux', 'np', 'os', 'phydra', 'phydra_setup', 'quit', 'sys', 'xs']


In [6]:
print('xarray-simlab version: ', xs.__version__)
print('phydra version: ', phydra.__version__)

xarray-simlab version:  0.4.1
phydra version:  0+untagged.70.ga52b5f6.dirty


In [7]:
xs.process_info(make_FX_flux(Sinking, 'S'))

xs.process_info(make_FX_flux(Upwelling, 'U'))

Sinking_S
<class 'phydra.processes.forcingfluxes.Sinking'> S
<Sinking_S  (xsimlab process)>
Variables:
    m                [in] <--- GekkoContext.m
    gk_context       [in] <--- GekkoContext.context
    gk_SVs           [in] <--- GekkoContext.SVs
    gk_SVshapes      [in] <--- GekkoContext.SVshapes
    gk_Fluxes        [in] <--- GekkoContext.Fluxes
    gk_Flux_Int      [in] <--- GekkoContext.Flux_Intermediates
    gridshape        [in] <--- GekkoContext.shape
    MLD              [in] <--- Slab.MLD
    MLD_deriv        [in] <--- Slab.MLD_deriv
    N0_forcing       [in] <--- Slab.N0_forcing
    kappa            [in] constant mixing coefficient
    fxflux_label    [out]
    fx_output       [out] ('S', 'time') 
    C_labels         [in] S label of component(s) that grows
    S               [out] ('S',) 
Simulation stages:
    initialize
    finalize_step
Upwelling_U
<class 'phydra.processes.forcingfluxes.Upwelling'> U
<Upwelling_U  (xsimlab process)>
Variables:
    m                [in

In [8]:
model = xs.Model({
    'time':Time, 'grid':Grid0D, 'ibcs':Boundary0D, 
    
    'context':GekkoContext,
                  
    'N':make_Component('Nutrient','N'), 'P':make_Component('Phytoplankton','P'),
    'Z':make_Component('Zooplankton','Z'), 'D':make_Component('Detritus','D'),
    'Env':Slab, 
    
    # Fluxes
    #'P_Growth':LimitedGrowth, 
    
    # ForcingFluxes
    'Sinking':make_FX_flux(Sinking, 'S'),
    'Upwelling':make_FX_flux(Upwelling, 'U'),
    
    'MLD':SinusoidalMLD, 
    'N0':ConstantN0, 
    
    'solver':GekkoSolve
})

model

Sinking_S
<class 'phydra.processes.forcingfluxes.Sinking'> S
Upwelling_U
<class 'phydra.processes.forcingfluxes.Upwelling'> U


<xsimlab.Model (14 processes, 17 inputs)>
context
time
    days            [in] ('time',) time in days
grid
    shape           [in]
    length          [in]
ibcs
    ibc             [in] Initial Boundary Conditions
N
    init            [in]
    dim             [in]
P
    init            [in]
    dim             [in]
Z
    dim             [in]
    init            [in]
D
    init            [in]
    dim             [in]
N0
    constant_N0     [in] value of the constant Nutrient forcing
MLD
Sinking
    C_labels        [in] S label of component(s) that grows
    kappa           [in] constant mixing coefficient
Upwelling
    kappa           [in] constant mixing coefficient
    C_labels        [in] U label of component(s) that grows
Env
solver

In [9]:
model.index_vars 


[('time', 'time'),
 ('N', 'N'),
 ('P', 'P'),
 ('Z', 'Z'),
 ('D', 'D'),
 ('Env', 'components'),
 ('Env', 'forcingfluxes'),
 ('Sinking', 'S'),
 ('Upwelling', 'U')]

In [10]:
in_ds = phydra_setup(
    model=model,
    input_vars={
        'time__days': ('time', np.arange(0, 365*3)), #365*
        
        'N':{'dim':1,'init':1}, 
        'P':{'dim':3,'init':1},
        'Z':{'dim':1,'init':1},
        'D':{'dim':1,'init':1},
        
        #'P_Growth':{'mu':1.,
        #       'C_label':'P',
        #       'R_label':'N',
        #       'halfsat':0.5},
        
        'Sinking':{'kappa':0.1,
               'C_labels':['D','P']},
        
        'Upwelling':{'kappa':0.1,
               'C_labels':['N']},
        
        'N0__constant_N0':2.
    },
    output_vars={
        'Env__comp_output':None,
        'Env__fxflux_output':None,
    }
                 )

In [ ]:
with model:
    out_ds = in_ds.xsimlab.run()

Initializing Gekko Context
Initializing Model Time
dimensions of component N are initialized at 1
Initializing component  N ['N']
FULLDIMS [[0.]]
WHUT [[0]]
[[1]]
dimensions of component P are initialized at 3
Initializing component  P ['P-0', 'P-1', 'P-2']
FULLDIMS [[0. 0. 0.]]
WHUT [[0 0 0]]
[[1 1 1]]
dimensions of component Z are initialized at 1
Initializing component  Z ['Z']
FULLDIMS [[0.]]
WHUT [[0]]
[[1]]
dimensions of component D are initialized at 1
Initializing component  D ['D']
FULLDIMS [[0.]]
WHUT [[0]]
[[1]]
dimensions of component Sinking are initialized at ['D' 'P']
Initializing forcing flux Sinking for components ['D' 'P']
D initFXflux [[0.0]]
D initFXflux [[0]]
P initFXflux [[0.0 0.0 0.0]]
P initFXflux [[0 0 0]]
dimensions of component Upwelling are initialized at ['N']
Initializing forcing flux Upwelling for components ['N']
N initFXflux [[0.0]]
N initFXflux [[0]]


Initializing Environment: 
 components:['N', 'P-0', 'P-1', 'P-2', 'Z', 'D'] 
 fx-fluxes:['Sinking-D',

In [ ]:
out_ds

In [ ]:
out_ds.Env__comp_output.plot.line(x='time')

In [ ]:
out_ds.Env__fxflux_output.plot.line(x='time')